modules

In [2]:
import pandas as pd
import time
import networkx as nx
import csv
import requests
from requests.exceptions import Timeout

constants and global variables

In [8]:
SAMEAS = "http://www.w3.org/2002/07/owl#sameAs"
MY_REDIRECT = "https://krr.triply.cc/krr/metalink/def/redirectedTo"
NOTFOUND = 1
NOREDIRECT = 2
ERROR = 3
TIMEOUT = 4
REDIRECT = 5
STANDARD_TIMEOUT = (0.01, 0.05)
RETRY_TIMEOUT = (0.5, 2.5)
FINAL_TRY_TIMEOUT = (1, 5)

files = [] # locations of the graphs

find redirects\
input: IRI, timeout (tuple)\
output: NOTFOUND etc, and the URLS it got redirected to (optional)\
go to IRI and if response is 404 return NOTFOUND, None\
if response url is IRI return NOREDIRECT, None\
else return REDIRECT, and all the urls it redirected to\
if there is no response url return NOREDIRECT, None\
return TIMEOUT, None in case of time out\
return ERROR, none in case of error

In [5]:
def find_redirects(iri, timeout):
    try:
        collect_urls = []
        response = requests.get(iri, timeout=timeout, allow_redirects=True)

        if response.status_code == 404:
            return NOTFOUND, None
        if response.history:
            if response.url == iri: # check if reponse.url encodes iri
                return NOREDIRECT, None
            else:
                for resp in response.history: # why loop through history here but not at respoonse.url == iri
                    collect_urls.append(resp.url)
                collect_urls.append(response.url) # check for duplicates
                return REDIRECT, collect_urls
    except Timeout:
        return TIMEOUT, None
    except:
        return ERROR, None

load entities\
input: path to the graph (CSV)\
output: set with all subjects and objects in the graph\
load entities from a csv, then loop through all subjects and objects and put them in a set

In [6]:
def load_entities(path_to_input_graph):
    input_graph_data = pd.read_csv(path_to_input_graph)

    sources = input_graph_data['SUBJECT']
    targets = input_graph_data['OBJECT']
    edge_data = zip(sources, targets)

    entities = set()
    for (s,t) in edge_data:
        entities.add(s)
        entities.add(t)
    return entities

TODO use Pool to get multiple IRI's

In [ ]:
from multiprocessing import pool
for file in files:
    print(file)
    start = time.time()

    redi_graph = nx.DiGraph()

    entities_to_test = load_entities(file)
    print(f"There are {len(entities_to_test)} entities in the graph")
    
    timeout_entities = set()
    original_entities = entities_to_test.copy()
    redirect_result = {}

    for e in entities_to_test:
        redirect_result[e] = None
    
    while len(entities_to_test) != 0:
        collect_new_entities_to_test = set()
        with pool(10) as pool:
            for e in pool.imap_unordered(find_redirects, entities_to_test):
                pass